In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16,preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator  ## For Image Augmentation
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
% matplotlib inline

In [ ]:
## Resizing all images to a default image size of VGG16
IMAGE_SIZE = [224,224]

In [ ]:
train_path = 'train_path'
test_path = 'test_path'

In [ ]:
vgg = VGG16(input_shape=IMAGE_SIZE+[3],weights='imagenet',include_top = False)

In [ ]:
vgg.layers  # Pretrained layers of VGG16

In [ ]:
for layer in vgg.layers:
    layer.trainable = False
    
## We donot need to train the model because the model is trained and thus we as only transfering 
## the trained model into our compatibility data.

In [ ]:
## To see the number of categories in this
folders = glob('Datasets/')
folders

In [ ]:
## Flattening
x = Flatten()(vgg.output)

In [ ]:
## Appending len(folders)(i.e number of outputs ) to last layers
prediction = Dense(len(folders),activation='softmax')(x)

In [ ]:
model = Model(inputs = vgg.inputs,outputs = prediction)
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('Datasets/Train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('Datasets/Test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')


In [ ]:
# fit the model
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

In [ ]:
# loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

In [ ]:
# accuracies
plt.plot(r.history['acc'], label='train acc')
plt.plot(r.history['val_acc'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
from keras.models import load_model
model.save('facefeatures_new_model.h5')